In [2]:
import simdjson as json
import copy
import os 

import numpy as np
from process_bigraph import pp
from vivarium.core.composer import Composer

from ecoli.experiments.ecoli_master_sim import EcoliSim
from ecoli.shared.registry import ecoli_core


def is_not_serializable(d):
    def has_non_serializable(d):
        import numpy as np
        """Returns a dict whose keys match that of d, except replacing innermost values (v) with their corresponding _default declarations.
        Used for migration.
        """
        if isinstance(d, dict):
            for key, val in d.items():
                if isinstance(val, np.ndarray) or type(val).__name__ == "function": 
                    return True
            else:
                return {k: has_non_serializable(v) for k, v in d.items()}
        else:
            return d
    has = has_non_serializable(d)
    if isinstance(has, dict):
        return False 
    else:
        return has 
    

def extract_process_states():
    sim = EcoliSim.from_file()
    sim.build_ecoli()
    processes = sim.processes

    data_dir = '/Users/alexanderpatrie/Desktop/repos/ecoli/v2Ecoli/data/model'
    topology_fp = os.path.join(data_dir, 'single_topology.json')
    with open(topology_fp, 'r') as f:
        raw_state = json.load(f)
    
    state = {}
    for process_id, process_ports in raw_state.items():
        process_config = {}
        if process_id in processes:
            # TODO: this needs to be serialized for arrays
            instance = processes[process_id]
            default_config = instance.defaults
            if not is_not_serializable(default_config):
                process_config = default_config

        state[process_id] = {
            "address": f"local:{process_id}",
            "config": process_config,
            "inputs": process_ports,
            "outputs": process_ports
        }
    
    # TODO: export doc here
    state_fp = os.path.join(data_dir, 'single_state.json')
    with open(state_fp, 'w') as f:
        json.dump(state, f, indent=4)

state = extract_process_states()
state

{'RNA_counts_listener': {'address': 'local:RNA_counts_listener',
  'config': {'rna_ids': [],
   'mrna_indexes': [],
   'time_step': 1,
   'emit_unique': False},
  'inputs': {'listeners': ['listeners'],
   'RNAs': ['unique', 'RNA'],
   'global_time': ['global_time'],
   'timestep': ['timestep']},
  'outputs': {'listeners': ['listeners'],
   'RNAs': ['unique', 'RNA'],
   'global_time': ['global_time'],
   'timestep': ['timestep']}},
 'bulk-timeline': {'address': 'local:bulk-timeline',
  'config': {'time_step': 1.0, 'timeline': []},
  'inputs': {'bulk': ['bulk'],
   'global': ['timeline'],
   'media_id': ['environment', 'media_id']},
  'outputs': {'bulk': ['bulk'],
   'global': ['timeline'],
   'media_id': ['environment', 'media_id']}},
 'dna_supercoiling_listener': {'address': 'local:dna_supercoiling_listener',
  'config': {'relaxed_DNA_base_pairs_per_turn': 0,
   'emit_unique': False,
   'time_step': 1},
  'inputs': {'listeners': ['listeners'],
   'chromosomal_segments': ['unique', 'chr

In [28]:
extract_process_states()

TypeError: Object of type ndarray is not JSON serializable

In [ ]:
def check(winning, x):
    return any

In [6]:
import numpy as np 


def collapse_defaults(d):
    """Returns a dict whose keys match that of d, except replacing innermost values (v) with their corresponding _default declarations.
    Used for migration.
    """
    if isinstance(d, dict):
        for key, val in d.items():
            if isinstance(val, np.ndarray):
                return val.tolist()
        else:
            return {k: collapse_defaults(v) for k, v in d.items()}
    else:
        return d


i = dict(zip(
    ['x', 'y', 'z'], 
    [{'value': np.random.random((11,))} for _ in range(3)]
))
j = dict(zip(
    ['x', 'y', 'z'], 
    [{'value': np.random.random((11,))} for _ in range(3)]
))
i.update(j)
v = copy.deepcopy(i)
nesting = 11
x = dict(zip(
    [str(np.random.random()) for _ in range(nesting)],
    [{str(k): v for k in range(nesting)}]
))
collapse_defaults(x)



{'0.21666411577972544': {'0': {'x': [0.30879803475824874,
    0.4520602625540353,
    0.5143131096597544,
    0.36937366723819254,
    0.4025407184816323,
    0.685127228523439,
    0.018310730681360354,
    0.04991313360417404,
    0.7560576571048745,
    0.9316870349333779,
    0.9025310222699645],
   'y': [0.2823299827098569,
    0.7381462903085189,
    0.5843982650690922,
    0.7246021245400406,
    0.6582871675509195,
    0.5124455506305847,
    0.9619968026986385,
    0.7050474302831918,
    0.47286811333042744,
    0.4151637066772794,
    0.8162880737867027],
   'z': [0.28709466499701053,
    0.10935615431438916,
    0.012330470679517425,
    0.28622333854440296,
    0.07532815265592296,
    0.1698593452088527,
    0.2755463267017322,
    0.19617032547050872,
    0.09174688412058718,
    0.9308033828520633,
    0.9279503967278848]},
  '1': {'x': [0.30879803475824874,
    0.4520602625540353,
    0.5143131096597544,
    0.36937366723819254,
    0.4025407184816323,
    0.6851272285